In [ ]:
#QUESTION 1:

#CREATE TABLE provider_licenses
CREATE TABLE provider_licenses (
    provider_id INT,
    states_of_license VARCHAR(100)
);

#Step 2: Insert Data
INSERT INTO provider_licenses (provider_id, states_of_license) VALUES
(321, 'NY, CA, FL'),
(5433, 'NY, CA'),
(31276, 'FL, CA'),
(43, 'DL'),
(543, 'DL'),
(432, 'TX'),
(54932, 'TX');



#TO FLATTEN THE DATA INTO ROWS

WITH Split AS (
    SELECT
        provider_id,
        TRIM(SPLIT_PART(states_of_license, ',', 1)) AS state_of_license,
        1 AS num_order
    FROM provider_licenses

UNION ALL
    SELECT
        provider_id,
        TRIM(SPLIT_PART(states_of_license, ',', 2)) AS state_of_license,
        2 AS num_order
    FROM provider_licenses
    WHERE SPLIT_PART(states_of_license, ',', 2) <> ''

UNION ALL
    SELECT
        provider_id,
        TRIM(SPLIT_PART(states_of_license, ',', 3)) AS state_of_license,
        3 AS num_order
    FROM provider_licenses
    WHERE SPLIT_PART(states_of_license, ',', 3) <> ''
)
SELECT provider_id, state_of_license
FROM Split
order by 1,2




In [ ]:
#QUESTION 2: WRITE SQL QUERY GENERATES  A MATRIX of Co-licenses following:
#NOTE THIS IS CONNECTED TO PREVIOUS FLATTEN CTE (Split)



SELECT
    state_1,
    SUM(CASE WHEN state_2 = 'CA' THEN cnt ELSE 0 END) AS CA,
    SUM(CASE WHEN state_2 = 'DL' THEN cnt ELSE 0 END) AS DL,
    SUM(CASE WHEN state_2 = 'FL' THEN cnt ELSE 0 END) AS FL,
    SUM(CASE WHEN state_2 = 'NY' THEN cnt ELSE 0 END) AS NY,
    SUM(CASE WHEN state_2 = 'TX' THEN cnt ELSE 0 END) AS TX
FROM (

 SELECT
        a.state_of_license AS state_1,
        b.state_of_license AS state_2,
        COUNT(DISTINCT a.provider_id) AS cnt
    FROM
        Split a
    JOIN
        Split b
    ON
        a.provider_id = b.provider_id
    GROUP BY
        1, 2
) AS CoLicenseMatrix
GROUP BY 1
ORDER BY 1;

In [ ]:
#QUESTION 3: GIVEN A TABLE L(i,j) to determine how many providers exist in the network


import pandas as pd

# Co-license matrix data
data = {
    'CA': [3, 0, 2, 2, 0],
    'DL': [0, 2, 0, 0, 0],
    'FL': [2, 0, 2, 1, 0],
    'NY': [2, 0, 1, 2, 0],
    'TX': [0, 0, 0, 0, 2]
}
index = ['CA', 'DL', 'FL', 'NY', 'TX']
df = pd.DataFrame(data, index=index)



unique_providers = set()


for i, state1 in enumerate(df.index):
    for j, state2 in enumerate(df.columns):
        count = df.iat[i, j]
        if count > 0:
            for k in range(count):
              #if only licensed in one state, we add just one state
                if i == j:
                    unique_providers.add((state1,))
              #otherwise add two states sorted
                else:
                    unique_providers.add(tuple(sorted([state1, state2])))

print(f"Number of unique providers: {len(unique_providers)}")


Number of unique providers: 8


In [ ]:
#QUESTION 4: Calculate the maximum number of clients that the network can accept in each state. Write down assumptions
#Providers maybe listed in multiple states
#We need to merge two dataframes (Capacity) AND (State_licenses) on provider_id


capacity = {
    "provider_id": [321, 5433, 31276, 43, 543, 432, 54932],
    "max_client_capacity": [30, 40, 40, 20, 10, 50, 30]
}

# Data from the second table
state_licenses = {
    "provider_id": [321, 321, 321, 5433, 5433, 31276, 31276, 43, 543, 432, 54932],
    "state_of_license": ["CA", "FL", "NY", "CA", "NY", "CA", "FL", "DL", "DL", "TX", "TX"]
}

# Create DataFrames for capacity and state licenses
df_capacity = pd.DataFrame(capacity)
df_state_licenses = pd.DataFrame(state_licenses)

# JOIN DataFrames on provider_id
df_merged = pd.merge(df_state_licenses, df_capacity, on="provider_id")

# Group by state and calculate sum of max_client_capacity
df_state_capacity = df_merged.groupby("state_of_license")["max_client_capacity"].sum().reset_index()

print(df_state_capacity)

#MAX CAPACITY CA: 110, DL:30, FL: 70, NY:70, and TX:80

  state_of_license  max_client_capacity
0               CA                  110
1               DL                   30
2               FL                   70
3               NY                   70
4               TX                   80


In [ ]:
#QUESTION 5:
import pandas as pd
import numpy as np

# INPUT data
data = {
    'State': ['NY', 'CA', 'FL', 'DL', 'TX'],
    'Projected number of clients': [400, 350, 300, 425, 600],
    'Current number of clients': [350, 250, 100, 400, 550],
    'Current avg. max provider capacity': [40, 50, 30, 30, 20]
}

# Convert the data into a DataFrame
df = pd.DataFrame(data)

#Calculate the deficit between Projects and Current Number
df['Client Deficit'] = df['Projected number of clients'] - df['Current number of clients']

#Divide the Client Deficit by Avg Max provider capacity - ROUND AND CONVERT TO INTEGER GIVEN WE care about # of providers
df['Additional Providers Needed'] = np.ceil(df['Client Deficit'] / df['Current avg. max provider capacity']).astype(int)

print(df[['State', 'Client Deficit', 'Additional Providers Needed']])

  State  Client Deficit  Additional Providers Needed
0    NY              50                            2
1    CA             100                            2
2    FL             200                            7
3    DL              25                            1
4    TX              50                            3


In [ ]:

#QUESTION 6:

import pandas as pd
import numpy as np

# Data Included + Added Growth Rates by State

data = {
    'State': ['NY', 'CA', 'FL', 'DL', 'TX'],
    'Projected number of clients': [400, 350, 300, 425, 600],
    'Current number of clients': [350, 250, 100, 400, 550],
    'Current avg. max provider capacity': [40, 50, 30, 30, 20],
    'Monthly growth rate': [0.01, 0.02, 0.02, 0.03, 0.01]
}

df = pd.DataFrame(data)

# Define the number of months to project growth (e.g., 12 months for a year)
# Calculate adjusted projected # of clients given the growth rates
months = 12
df['Adjusted Projected Clients'] = df['Projected number of clients'] * ((1 + df['Monthly growth rate']) ** months)

# Calculate the client deficit updated
df['Client Deficit Updated'] = df['Adjusted Projected Clients'] - df['Current number of clients']

# Calculate additional providers needed
df['Additional Providers Needed'] = np.ceil(df['Client Deficit Updated'] / df['Current avg. max provider capacity']).astype(int)

#Print Results
print(df[['State', 'Client Deficit Updated', 'Additional Providers Needed']])

#COMPARED TO SOLUTION 5:

#NY 2 -> 3
#CA 2 -> 4
#FL 7 -> 10
#DL 1 -> 7
#TX 3 -> 7


  State  Client Deficit Updated  Additional Providers Needed
0    NY              100.730012                            3
1    CA              193.884628                            4
2    FL              280.472538                           10
3    DL              205.948377                            7
4    TX              126.095018                            7
